In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from matplotlib.pyplot import plot, show
from sklearn.metrics import roc_auc_score
import json
from gensim.models.word2vec import Word2Vec

# 1.导入数据

In [4]:
df_train = pd.read_csv('data/train_dataset.csv', sep='\t')
df_test = pd.read_csv('data/test_dataset.csv', sep='\t')
sub = pd.read_csv('data/submit_example.csv')
df_test['id'] = sub['id']
df = pd.concat([df_train, df_test])

# 2.细化特征

In [5]:
df['location_first_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['first_lvl'])
df['location_sec_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['sec_lvl'])
df['location_third_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['third_lvl'])

# 构建训练特征
feats = ['user_name', 'action', 'auth_type', 'ip_location_type_keyword', 'ip_risk_level', 'ip', 'location',
         'device_model', 'os_type', 'os_version', 'browser_type', 'browser_version', 'bus_system_code', 'op_target',
         'location_first_lvl', 'location_sec_lvl', 'location_third_lvl',
         ]

cat = []

LABEL = 'risk_label'

# 细分时间特征
df['sec'] = df['session_id'].apply(lambda x: int(x[-7:-5]))
df['sec_sin'] = np.sin(df['sec'] / 60 * 2 * np.pi)
df['sec_cos'] = np.cos(df['sec'] / 60 * 2 * np.pi)
df['op_date'] = pd.to_datetime(df['op_date'])
df['hour'] = df['op_date'].dt.hour
df['weekday'] = df['op_date'].dt.weekday
df['year'] = df['op_date'].dt.year
df['month'] = df['op_date'].dt.month
df['day'] = df['op_date'].dt.day
df['op_ts'] = df["op_date"].values.astype(np.int64) // 10 ** 9
df = df.sort_values(by=['user_name', 'op_ts']).reset_index(drop=True)
df['last_ts'] = df.groupby(['user_name'])['op_ts'].shift(1)
df['last_ts2'] = df.groupby(['user_name'])['op_ts'].shift(2)
df['ts_diff'] = df['op_ts'] - df['last_ts']
df['ts_diff2'] = df['op_ts'] - df['last_ts2']
feats += ['sec',
          'sec_sin', 'sec_cos',
          'op_ts', 'last_ts', 'ts_diff',
          # 'last_ts2',
          'ts_diff2',
          ]

# 3.词嵌入特征

In [6]:
# 词嵌入特征
for name in ['auth_type']:
    df[name + '_fillna'] = df[name].astype('str')
    sent = df.groupby(['user_name', 'year', 'month', 'day'])[name + '_fillna'].agg(list).values

    vec_size = 6
    w2v_model = Word2Vec(sentences=sent, vector_size=vec_size, window=12, min_count=1, workers=1)
    tmp = df[name + '_fillna'].map(lambda x: w2v_model.wv[x])
    tmp = pd.DataFrame(list(tmp))
    tmp.columns = ['_'.join([name, 'emb', str(i)]) for i in range(vec_size)]
    df = pd.concat([df, tmp], axis=1)
    feats += list(tmp.columns)

# for name in df['auth_type']:
for w in w2v_model.wv.key_to_index:
    print(w, w2v_model.wv[w])

# 特征构造
for name in ['mean', 'std', 'max', 'min', 'median', 'skew']:
    for name1 in ['user_name', 'bus_system_code', 'auth_type', 'action',
                  ]:  # 'op_target'

        df[name1 + '_ts_diff_' + name] = df.groupby([name1])['ts_diff'].transform(name)
        feats.append(name1 + '_ts_diff_' + name)

df['if_out'] = (df['location'] == '{"first_lvl":"成都分公司","sec_lvl":"9楼","third_lvl":"销售部"}')
feats.append('if_out')

nan [-0.13903026  0.03005757  0.25148448  0.9644866  -0.28805935 -0.67375904]
pwd [ 0.03973053  0.1891723   0.13216272  1.0011268  -0.16735148 -0.7792582 ]
sms [-0.12732314  0.05897151  0.15256187  1.0156738  -0.19783318 -0.69883054]
qr [-0.15083304 -0.18690944  0.32928893  1.0530428  -0.14471556 -0.6634975 ]
otp [ 0.10661024 -0.01189896  0.15297233  0.95404774 -0.40966824 -0.6917451 ]


# 4.标签编码

In [7]:
# 标签编码
for name in ['user_name', 'action', 'auth_type', 'ip', 'ip_location_type_keyword', 'ip_risk_level', 'location',
             'device_model', 'os_type', 'os_version', 'browser_type', 'browser_version', 'bus_system_code',
             'op_target',
             'location_first_lvl', 'location_sec_lvl', 'location_third_lvl',
             ] + cat:
    le = LabelEncoder()
    df[name] = le.fit_transform(df[name])

# 5.分离训练集和测试集

In [8]:
df_train = df[~df[LABEL].isna()].reset_index(drop=True)
df_test = df[df[LABEL].isna()].reset_index(drop=True)

# 6.确定模型参数

In [9]:
params = {
    'learning_rate': 0.08,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
}
feats

['user_name',
 'action',
 'auth_type',
 'ip_location_type_keyword',
 'ip_risk_level',
 'ip',
 'location',
 'device_model',
 'os_type',
 'os_version',
 'browser_type',
 'browser_version',
 'bus_system_code',
 'op_target',
 'location_first_lvl',
 'location_sec_lvl',
 'location_third_lvl',
 'sec',
 'sec_sin',
 'sec_cos',
 'op_ts',
 'last_ts',
 'ts_diff',
 'ts_diff2',
 'auth_type_emb_0',
 'auth_type_emb_1',
 'auth_type_emb_2',
 'auth_type_emb_3',
 'auth_type_emb_4',
 'auth_type_emb_5',
 'user_name_ts_diff_mean',
 'bus_system_code_ts_diff_mean',
 'auth_type_ts_diff_mean',
 'action_ts_diff_mean',
 'user_name_ts_diff_std',
 'bus_system_code_ts_diff_std',
 'auth_type_ts_diff_std',
 'action_ts_diff_std',
 'user_name_ts_diff_max',
 'bus_system_code_ts_diff_max',
 'auth_type_ts_diff_max',
 'action_ts_diff_max',
 'user_name_ts_diff_min',
 'bus_system_code_ts_diff_min',
 'auth_type_ts_diff_min',
 'action_ts_diff_min',
 'user_name_ts_diff_median',
 'bus_system_code_ts_diff_median',
 'auth_type_ts_di

# 7.使用n折交叉验证划分数据集（此处n取10）训练模型并进行预测

In [10]:
seeds = [2022]
# 初始化oof数组
oof = np.zeros(len(df_train))
importance = 0
fold_num = 10
pred_y = pd.DataFrame()
for seed in seeds:
    print('############################', seed)

    # 10折交叉验证划分数据
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[LABEL])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, LABEL])
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, LABEL])
        model = lgb.train(params, train, valid_sets=val, num_boost_round=10000,
                          early_stopping_rounds=100, verbose_eval=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num

df_train['oof'] = oof
score = roc_auc_score(df_train[LABEL], df_train['oof'])
score

############################ 2022
----------- 0
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.53474
Early stopping, best iteration is:
[39]	valid_0's auc: 0.54762


C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


----------- 1
Training until validation scores don't improve for 100 rounds


C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.493507
Early stopping, best iteration is:
[1]	valid_0's auc: 0.511404
----------- 2
Training until validation scores don't improve for 100 rounds


...

C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.502106
Early stopping, best iteration is:
[13]	valid_0's auc: 0.522478
----------- 9


C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\wgq\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.509078
Early stopping, best iteration is:
[6]	valid_0's auc: 0.5348


0.5325646050072894

# **8.获取特征重要性**

In [11]:
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
feats_importance.sort_values('importance', ascending=False)[:10]

,name,importance
23,ts_diff2,571.175817
20,op_ts,546.814387
22,ts_diff,510.694570
19,sec_cos,307.498871
18,sec_sin,304.902033
21,last_ts,301.187301
17,sec,280.897311
50,user_name_ts_diff_skew,134.404960
12,bus_system_code,104.902371
0,user_name,100.380022


# 9.保存结果

In [12]:
sub = pd.read_csv('data/submit_example.csv')
pred_y = pred_y.mean(axis=1)
sub['ret'] = pred_y
sub[['id', 'ret']].to_csv('ans/lgb_5285.csv', index=False)